# 1. Importaciones

In [ ]:
import numpy as np
import pandas as pd
import random
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler

# 2. Generando Dataset

In [ ]:
import random
import numpy as np
import pandas as pd

random.seed(222)
np.random.seed(222)


# CONFIGURACIÓN MEJORADA

valor_plan = {
    'basico': 4.99,
    'estandar': 9.99,
    'premium': 14.99
}

data = []
n = 5000

for _ in range(n):
    plan = random.choices(['basico', 'estandar', 'premium'], weights=[0.3, 0.5, 0.2])[0]
    antiguedad = random.randint(1, 60)
    facturas_impagas = random.randint(0, 3)
    frecuencia_uso = random.randint(0, 30)
    metodo_pago = random.choice(['transferencia_bancaria', 'tarjeta_credito', 'tarjeta_debito', 'efectivo'])
    tickets_soporte = random.randint(0, 10)
    tipo_contrato = random.choices(['mensual', 'anual'], weights=[0.7, 0.3])[0]
    cambios_plan = random.randint(0, 3)
    canal_adquisicion = random.choice(['web', 'referido', 'redes_sociales', 'call_center'])


    # PROBABILIDAD BASE MÁS ALTA

    prob_churn = 0.15  # 15% base (antes 3%)


    # ANTIGUEDAD (señal MÁS FUERTE)

    if antiguedad < 6:
        prob_churn += 0.25  # +25% (antes +9%)
    elif antiguedad < 12:
        prob_churn += 0.15  # +15% (antes +5%)
    elif antiguedad > 36:
        prob_churn -= 0.30  # -30% (antes -10%)

    # PLAN (diferencias más marcadas)

    if plan == 'basico':
        prob_churn += 0.15  # +15% (antes +5%)
    elif plan == 'premium':
        prob_churn -= 0.25  # -25% (antes -8%)


    # MÉTODO DE PAGO (señal más fuerte)

    if metodo_pago == 'efectivo':
        prob_churn += 0.20  # +20% (antes +10%)
    elif metodo_pago == 'transferencia_bancaria':
        prob_churn += 0.10  # +10% (antes +5%)
    else:
        prob_churn -= 0.15  # -15% (antes -8%)


    # FACTURAS IMPAGAS (señal CRÍTICA)

    if facturas_impagas == 0:
        prob_churn -= 0.25  # -25% (antes -18%)
    elif facturas_impagas == 1:
        prob_churn += 0.15  # +15% (antes +7%)
    elif facturas_impagas >= 2:
        prob_churn += 0.35  # +35% (antes +16%) ← SEÑAL FUERTE


    # FRECUENCIA DE USO (más impacto)

    if frecuencia_uso < 5:
        prob_churn += 0.25  # +25% (antes +10%)
    elif frecuencia_uso < 10:
        prob_churn += 0.12  # +12% (antes +5%)
    elif frecuencia_uso > 20:
        prob_churn -= 0.20  # -20% (antes -10%)


    # TICKETS DE SOPORTE (más diferenciación)

    if tickets_soporte == 0:
        prob_churn -= 0.10  # -10% (antes -6%)
    elif tickets_soporte > 5:
        prob_churn += 0.25  # +25% (antes +12%)
    elif tickets_soporte > 2:
        prob_churn += 0.12  # +12% (antes +6%)


    # TIPO DE CONTRATO (más peso)

    if tipo_contrato == 'anual':
        prob_churn -= 0.20  # -20% (antes -10%)
    else:
        prob_churn += 0.15  # +15% (antes +10%)


    # CAMBIOS DE PLAN (señal de inestabilidad)

    if cambios_plan >= 2:
        prob_churn += 0.25  # +25% (antes +12%)
    elif cambios_plan == 1:
        prob_churn += 0.10  # +10% (antes +6%)


    # CANAL DE ADQUISICIÓN (más diferenciado)

    if canal_adquisicion == 'referido':
        prob_churn -= 0.15  # -15% (antes -8%)
    elif canal_adquisicion == 'call_center':
        prob_churn += 0.12  # +12% (antes +6%)


    # INTERACCIONES (NUEVO - crea patrones más claros)


    # Combo MORTAL 1: Cliente nuevo + facturas impagas
    if antiguedad < 6 and facturas_impagas >= 2:
        prob_churn += 0.20  # Boost adicional

    # Combo PROTECTOR 1: Cliente antiguo + frecuencia alta
    if antiguedad > 36 and frecuencia_uso > 20:
        prob_churn -= 0.15  # Protección adicional

    # Combo MORTAL 2: Bajo uso + muchos tickets
    if frecuencia_uso < 5 and tickets_soporte > 5:
        prob_churn += 0.15  # Cliente frustrado

    # Combo PROTECTOR 2: Plan premium + contrato anual
    if plan == 'premium' and tipo_contrato == 'anual':
        prob_churn -= 0.10  # Cliente comprometido


    # LIMITAR PROBABILIDAD (rango más amplio)

    prob_churn = min(max(prob_churn, 0.02), 0.98)  # Antes: (0.01, 0.95)

    churn = random.choices([0, 1], weights=[1 - prob_churn, prob_churn])[0]


    # CREAR REGISTRO

    registro = {
        'antiguedad': antiguedad,
        'plan': plan,
        'metodo_pago': metodo_pago,
        'facturas_impagas': facturas_impagas,
        'frecuencia_uso': frecuencia_uso,
        'tickets_soporte': tickets_soporte,
        'tipo_contrato': tipo_contrato,
        'cambios_plan': cambios_plan,
        'canal_adquisicion': canal_adquisicion,
        'churn': churn
    }

    data.append(registro)


# CREAR Y GUARDAR DATASET

df = pd.DataFrame(data)
df.to_csv('dataset_sucio_v5.csv', index=False)

print("="*70)
print("✅ DATASET GENERADO CON SEÑALES FUERTES")
print("="*70)
print(f"Shape: {df.shape}")
print(f"\n📊 Distribución de Churn:")
print(df['churn'].value_counts(normalize=True))
print(f"\n📋 Columnas:")
print(df.columns.tolist())

# Análisis rápido
print(f"\n🔍 ANÁLISIS PRELIMINAR:")
print(f"Churn rate: {df['churn'].mean()*100:.2f}%")
print(f"Registros totales: {len(df)}")
print(f"Registros con churn: {df['churn'].sum()}")
print(f"Registros sin churn: {(df['churn']==0).sum()}")

✅ DATASET GENERADO CON SEÑALES FUERTES
Shape: (5000, 10)

📊 Distribución de Churn:
churn
1    0.5188
0    0.4812
Name: proportion, dtype: float64

📋 Columnas:
['antiguedad', 'plan', 'metodo_pago', 'facturas_impagas', 'frecuencia_uso', 'tickets_soporte', 'tipo_contrato', 'cambios_plan', 'canal_adquisicion', 'churn']

🔍 ANÁLISIS PRELIMINAR:
Churn rate: 51.88%
Registros totales: 5000
Registros con churn: 2594
Registros sin churn: 2406


# 3. Script para ensuciar Dataset

In [ ]:
def ensuciar_dataset(df, nivel='medio'):
    """
    Introduce valores nulos, outliers y ruido de forma controlada.
    """

    df_sucio = df.copy()

    porcentajes = {
        'bajo': {'nulos': 0.05, 'outliers': 0.03, 'ruido_label': 0.01},
        'medio': {'nulos': 0.10, 'outliers': 0.05, 'ruido_label': 0.02},
        'alto': {'nulos': 0.20, 'outliers': 0.10, 'ruido_label': 0.05}
    }

    config = porcentajes[nivel]
    n = len(df_sucio)

    print("="*70)
    print(f"🔧 ENSUCIANDO DATASET (Nivel: {nivel.upper()})")
    print("="*70)

    # 1. VALORES NULOS
    print(f"\n🔸 Introduciendo {config['nulos']*100}% valores nulos...")
    columnas_nulos = ['metodo_pago', 'facturas_impagas', 'frecuencia_uso', 'tickets_soporte', 'canal_adquisicion']

    for col in columnas_nulos:
        n_nulos = int(n * config['nulos'])
        indices_nulos = np.random.choice(df_sucio.index, size=n_nulos, replace=False)
        df_sucio.loc[indices_nulos, col] = np.nan
        print(f"   ↳ {col}: {n_nulos} nulos")

    # 2. OUTLIERS
    print(f"\n🔸 Introduciendo {config['outliers']*100}% outliers...")

    # Antiguedad: negativos o muy altos
    n_out = int(n * config['outliers'])
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'antiguedad'] = np.random.choice([-1, 0, 120, 150], size=n_out)
    print(f"   ↳ antiguedad: {n_out} outliers")

    # Frecuencia_uso: negativos o imposibles
    n_out = int(n * config['outliers'])
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'frecuencia_uso'] = np.random.choice([-5, -1, 50, 100], size=n_out)
    print(f"   ↳ frecuencia_uso: {n_out} outliers")

    # Tickets_soporte: muy altos
    n_out = int(n * config['outliers'])
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'tickets_soporte'] = np.random.randint(20, 100, size=n_out)
    print(f"   ↳ tickets_soporte: {n_out} outliers")

    # Facturas_impagas: negativos o muy altos
    n_out = int(n * config['outliers'] / 2)
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'facturas_impagas'] = np.random.choice([-1, 15, 20], size=n_out)
    print(f"   ↳ facturas_impagas: {n_out} outliers")

    # 3. RUIDO EN LABELS (CRÍTICO)
    print(f"\n🔸 Introduciendo {config['ruido_label']*100}% ruido en etiquetas...")
    n_ruido = int(n * config['ruido_label'])
    indices = np.random.choice(df_sucio.index, size=n_ruido, replace=False)
    df_sucio.loc[indices, 'churn'] = 1 - df_sucio.loc[indices, 'churn']
    print(f"   ↳ churn: {n_ruido} etiquetas invertidas (0↔1)")

    # 4. INCONSISTENCIAS LÓGICAS
    print(f"\n🔸 Introduciendo inconsistencias lógicas...")

    # Alto uso + muchas facturas impagas
    mask = df_sucio['frecuencia_uso'] > 25
    if mask.sum() > 0:
        sample = df_sucio[mask].sample(n=min(30, mask.sum()))
        df_sucio.loc[sample.index, 'facturas_impagas'] = 3
        print(f"   ↳ {len(sample)} registros: alto uso + deudas")

    # Plan premium + muchos tickets
    mask = df_sucio['plan'] == 'premium'
    if mask.sum() > 0:
        sample = df_sucio[mask].sample(n=min(20, mask.sum()))
        df_sucio.loc[sample.index, 'tickets_soporte'] = np.random.randint(8, 15, size=len(sample))
        print(f"   ↳ {len(sample)} registros: premium + muchos tickets")

    # 5. VALORES CATEGÓRICOS INVÁLIDOS
    print(f"\n🔸 Introduciendo valores categóricos inválidos...")

    n_inv = int(n * 0.01)
    indices = np.random.choice(df_sucio.index, size=n_inv, replace=False)
    df_sucio.loc[indices, 'plan'] = np.random.choice(['Premium', 'BASICO', 'standard', 'vip'], size=n_inv)
    print(f"   ↳ plan: {n_inv} valores incorrectos")

    indices = np.random.choice(df_sucio.index, size=n_inv, replace=False)
    df_sucio.loc[indices, 'metodo_pago'] = np.random.choice(['Tarjeta Credito', 'EFECTIVO', 'paypal'], size=n_inv)
    print(f"   ↳ metodo_pago: {n_inv} valores incorrectos")

    # RESUMEN
    total_nulos = df_sucio.isnull().sum().sum()
    print(f"\n{'='*70}")
    print(f"✅ DATASET ENSUCIADO")
    print(f"{'='*70}")
    print(f"Total valores nulos: {total_nulos}")
    print(f"Porcentaje nulos: {(total_nulos / (n * len(df_sucio.columns)))*100:.2f}%")

    return df_sucio

# EJECUTAR
df_limpio = pd.read_csv('dataset_churn_final.csv')
df_sucio = ensuciar_dataset(df_limpio, nivel='medio')
df_sucio.to_csv('dataset_churn_sucio.csv', index=False)

print("\n💾 Dataset guardado: dataset_churn_sucio.csv")

🔧 ENSUCIANDO DATASET (Nivel: MEDIO)

🔸 Introduciendo 10.0% valores nulos...
   ↳ metodo_pago: 500 nulos
   ↳ facturas_impagas: 500 nulos
   ↳ frecuencia_uso: 500 nulos
   ↳ tickets_soporte: 500 nulos
   ↳ canal_adquisicion: 500 nulos

🔸 Introduciendo 5.0% outliers...
   ↳ antiguedad: 250 outliers
   ↳ frecuencia_uso: 250 outliers
   ↳ tickets_soporte: 250 outliers
   ↳ facturas_impagas: 125 outliers

🔸 Introduciendo 2.0% ruido en etiquetas...
   ↳ churn: 100 etiquetas invertidas (0↔1)

🔸 Introduciendo inconsistencias lógicas...
   ↳ 30 registros: alto uso + deudas
   ↳ 20 registros: premium + muchos tickets

🔸 Introduciendo valores categóricos inválidos...
   ↳ plan: 50 valores incorrectos
   ↳ metodo_pago: 50 valores incorrectos

✅ DATASET ENSUCIADO
Total valores nulos: 2424
Porcentaje nulos: 4.85%

💾 Dataset guardado: dataset_churn_sucio.csv


In [ ]:
df_sucio.head(15)

,antiguedad,plan,metodo_pago,facturas_impagas,frecuencia_uso,tickets_soporte,tipo_contrato,cambios_plan,canal_adquisicion,churn
0,16,estandar,NaN,NaN,9.0,NaN,mensual,3,call_center,1
1,7,estandar,efectivo,3.0,2.0,10.0,mensual,2,referido,1
2,29,premium,tarjeta_credito,1.0,28.0,NaN,anual,3,web,0
3,48,estandar,tarjeta_debito,0.0,9.0,2.0,mensual,1,NaN,0
4,13,estandar,tarjeta_credito,1.0,19.0,5.0,mensual,0,web,0
5,2,estandar,tarjeta_credito,2.0,0.0,5.0,anual,3,redes_sociales,1
6,4,estandar,tarjeta_debito,2.0,25.0,NaN,anual,3,referido,1
7,45,premium,NaN,2.0,10.0,2.0,mensual,0,call_center,0
8,13,basico,efectivo,1.0,24.0,7.0,anual,0,web,0
9,49,basico,transferencia_bancaria,1.0,NaN,4.0,anual,3,call_center,0


# 4. Evaluo % de Churn en Dataset - Variable Target


In [ ]:
tabla_proporciones = df_sucio['churn'].value_counts(normalize=True).to_frame(name='proportion')
print(tabla_proporciones)

       proportion
churn            
1          0.5156
0          0.4844


# 5. Guardo el Dataset generado


In [ ]:
df_sucio.to_csv('dataset_sucio_v5.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
